# Analyzing IMDB Data in Keras
In this project, I will analyse a dataset from IMDB and use it to predict the sentiment analysis of a review.

In [1]:
# Imports
import numpy as np
import keras
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(42)

## 1. Loading the data
Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).

This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a list of word indexes (integers). For convenience, words are indexed by overall frequency in the dataset, so that for instance the integer "3" encodes the 3rd most frequent word in the data. This allows for quick filtering operations such as: "only consider the top 10,000 most common words, but eliminate the top 20 most common words".
As a convention, "0" does not stand for a specific word, but instead is used to encode the pad token.

This dataset comes preloaded with [Keras](https://keras.io/api/datasets/imdb/), so one simple command will get me training and testing data. There is a parameter for how many words I want to look at. For now, I will set it at 5000 (but this could be fine-tuned later on).

In [14]:
# Loading the data (it's preloaded in Keras)
number_of_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=number_of_words)

print(x_train.shape)
print(x_test.shape)

(25000,)
(25000,)


## 2. Examining the data
The data has been already pre-processed, where all the words have numbers, and the reviews come in as a vector with the words that the review contains. For example, if the word 'the' is the first one in the dictionary, and a review contains the word 'the', then there is a 1 in the corresponding vector.

The output comes as a vector of 1's and 0's, where 1 is a positive sentiment for the review, and 0 is negative.

In [15]:
print(x_train[0])
print(y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]
1


## 3. One-hot encoding the output
The data needs to be one-hot encoded in order to be ingested by the neural network (i.e. the input vectors are turned into (0,1)-vectors). For example, if the pre-processed vector contains the number 14, then in the processed vector, the 14th entry will be 1. Each entry below contains the list of 5000 characters used in this project, specifying whether or not they appear in each of the IMDB reviews.

In [16]:
# One-hot encoding the output into vector mode, each of length 1000
tokenizer = Tokenizer(num_words=number_of_words)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train[0])

[0. 1. 1. ... 0. 0. 0.]


The output also needs to be one-hot encoded. For example, if the output is 1, then the one-hot encoded output will be (0,1). If the output is 0, then the one-hot encoded output will be (1,0). For this, I will use the [to_categorical](https://keras.io/api/utils/python_utils/#to_categorical-function) function from Keras.

In [18]:
# One-hot encoding the output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 2)
(25000, 2)


## 4. Building the  model architecture
I will use a sequential model with three hidden layers. The hidden layers will have 1024, 512 nodes, and 64 nodes, respectively. The output layer will have 2 nodes, corresponding to the one-hot encoded output. I will use the [relu](https://keras.io/api/layers/activations/#relu-function) activation function for the first two layers, the [sigmoid](https://keras.io/api/layers/activations/#sigmoid-function) for the second layer and the [softmax](https://keras.io/api/layers/activations/#softmax-function) activation function for the output layer. I will use the [Adam](https://keras.io/api/optimizers/adam/) optimizer, and the [categorical_crossentropy](https://keras.io/api/losses/probabilistic_losses/#categoricalcrossentropy-class) loss function.

In [21]:
# Building the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_dim=number_of_words))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

# Compiling the model using categorical_crossentropy loss, and rmsprop optimizer.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])



Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 1024)              5121024   
                                                                 
 dropout_6 (Dropout)         (None, 1024)              0         
                                                                 
 dense_10 (Dense)            (None, 512)               524800    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 64)                32832     
                                                                 
 dropout_8 (Dropout)         (None, 64)                0         
                                                                 
 dense_12 (Dense)            (None, 2)                

## 5. Training the model
I will train the model for 10 epochs and a batch size of 32 and the data will be validated along the way with the dedicated test data.

In [22]:
# Running and evaluating the model
history = model.fit(x_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(x_test, y_test), 
          verbose=2)

Epoch 1/10
782/782 - 116s - loss: 0.3517 - accuracy: 0.8473 - val_loss: 0.2898 - val_accuracy: 0.8810 - 116s/epoch - 148ms/step
Epoch 2/10
782/782 - 105s - loss: 0.2296 - accuracy: 0.9108 - val_loss: 0.3042 - val_accuracy: 0.8780 - 105s/epoch - 134ms/step
Epoch 3/10
782/782 - 87s - loss: 0.1569 - accuracy: 0.9401 - val_loss: 0.4034 - val_accuracy: 0.8727 - 87s/epoch - 111ms/step
Epoch 4/10
782/782 - 83s - loss: 0.0883 - accuracy: 0.9682 - val_loss: 0.4554 - val_accuracy: 0.8727 - 83s/epoch - 107ms/step
Epoch 5/10
782/782 - 85s - loss: 0.0558 - accuracy: 0.9816 - val_loss: 0.5312 - val_accuracy: 0.8720 - 85s/epoch - 109ms/step
Epoch 6/10
782/782 - 95s - loss: 0.0396 - accuracy: 0.9875 - val_loss: 0.5724 - val_accuracy: 0.8715 - 95s/epoch - 122ms/step
Epoch 7/10
782/782 - 94s - loss: 0.0302 - accuracy: 0.9908 - val_loss: 0.5806 - val_accuracy: 0.8708 - 94s/epoch - 120ms/step
Epoch 8/10
782/782 - 94s - loss: 0.0244 - accuracy: 0.9924 - val_loss: 0.6033 - val_accuracy: 0.8688 - 94s/epoch -

Looking at the accuracy and loss plots, it seems that the model is overfitting. However, the accuracy is still quite high, so I will leave it as is for now.
## 6. Evaluating the model
Lastly, the accuracy of the model will be evaluated using the test data.

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: ", score[1])

Accuracy:  0.8539599776268005
